<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto City</font></h1>

## Introduction

This lab is a task for the Coursera Data Science specialization.

Author: Santiago Maraggi

Ref Link: https://medium.com/@dougm_9851/the-battle-of-neighborhoods-coursera-ibm-capstone-project-52b4292ef410

## Note for reviewers:
The article on wikipedia I found about Toronto Neighborhoods did not include geographical information, specifically, neighborhood coordinates: https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto

#### Unfortunately, the link to the proper wikipedia article was NOT in the INSTRUCTIONS section of the task to complete, so I found it after completing the task. Thank you for your understanding, if it is possible to have consideration for the case.

In order to accomplish this task, and following the spirit of the course, to remain adaptable to different circumstances, a new dataset was reached within official information services of the Toronto city.


#### IBM/Coursera Statement used as guide: "What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project."

The notebook downloads these datasets as needed. Also, some queries might be cached locally to enhance test repetitions and avoid saturating the free calls to the Foursquare API.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in New York City</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    sci

### 0. Download and explore TORONTO Datasets

## Links
Neighbourhoods: https://open.toronto.ca/dataset/neighbourhoods/

Neighbourhoods Profiles: https://open.toronto.ca/dataset/neighbourhood-profiles/

In [2]:
import urllib.request, json

# load neigbourhoods geojson in variable 'toronto_data_geojson'
with urllib.request.urlopen("https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/a083c865-6d60-4d1d-b6c6-b0c8a85f9c15?format=geojson&projection=4326") as url:
    toronto_data = json.loads(url.read().decode())

# save geojson locally for backup purposes
with open('toronto_neighbourhoods_data.geojson', 'w') as torontoneighborhoodsgeojsonfile:
      json.dump(toronto_data, torontoneighborhoodsgeojsonfile)

neighborhoods_data = toronto_data['features']
for feature in neighborhoods_data:
    print (feature['properties']['AREA_NAME'])

Wychwood (94)
Yonge-Eglinton (100)
Yonge-St.Clair (97)
York University Heights (27)
Yorkdale-Glen Park (31)
Lambton Baby Point (114)
Lansing-Westgate (38)
Lawrence Park North (105)
Lawrence Park South (103)
Leaside-Bennington (56)
Little Portugal (84)
Long Branch (19)
Malvern (132)
Maple Leaf (29)
Markland Wood (12)
Milliken (130)
Mimico (includes Humber Bay Shores) (17)
Morningside (135)
Moss Park (73)
Mount Dennis (115)
Mount Olive-Silverstone-Jamestown (2)
Mount Pleasant East (99)
Mount Pleasant West (104)
New Toronto (18)
Newtonbrook East (50)
Newtonbrook West (36)
Niagara (82)
North Riverdale (68)
North St.James Town (74)
O'Connor-Parkview (54)
Oakridge (121)
Oakwood Village (107)
Old East York (58)
Palmerston-Little Italy (80)
Parkwoods-Donalda (45)
Pelmo Park-Humberlea (23)
Playter Estates-Danforth (67)
Pleasant View (46)
Princess-Rosethorn (10)
Regent Park (72)
Rexdale-Kipling (4)
Rockcliffe-Smythe (111)
Roncesvalles (86)
Rosedale-Moore Park (98)
Rouge (131)
Runnymede-Bloor Wes

In [3]:
# dataframe column names
column_names = ['AREA_ID', 'AREA_LONG_CODE', 'AREA_NAME', 'LATITUDE', 'LONGITUDE'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

neighborhoods_data[0]

{'type': 'Feature',
 'properties': {'_id': 2661,
  'AREA_ID': 25886861,
  'AREA_ATTR_ID': 25926662,
  'PARENT_AREA_ID': 49885,
  'AREA_SHORT_CODE': 94,
  'AREA_LONG_CODE': 94,
  'AREA_NAME': 'Wychwood (94)',
  'AREA_DESC': 'Wychwood (94)',
  'X': None,
  'Y': None,
  'LONGITUDE': -79.425514947,
  'LATITUDE': 43.6769192679,
  'OBJECTID': 16491505,
  'Shape__Area': 3217959.609375,
  'Shape__Length': 7515.779658331329},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-79.43591570873059, 43.68015339477487],
    [-79.43491506339724, 43.68036886994886],
    [-79.43394727223847, 43.680578504490335],
    [-79.43388162422195, 43.68058996121469],
    [-79.43281544978878, 43.680808044458765],
    [-79.4326971769691, 43.68079658822322],
    [-79.43253064659865, 43.68082785962871],
    [-79.43245944449654, 43.68085882710364],
    [-79.43241132723996, 43.68089794537263],
    [-79.43094766397957, 43.681205221508016],
    [-79.429092835633, 43.681595102987544],
    [-79.42903365882387, 43.6814496

<a id='item1'></a>

## 1. Download and Explore Dataset

#### Load and explore the data

Next, let's load the data.

Let's take a quick look at the data.

Notice how all the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

#### Tranform the data into a *pandas* dataframe

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

Take a look at the empty dataframe to confirm that the columns are as intended.

In [4]:
neighborhoods

,AREA_ID,AREA_LONG_CODE,AREA_NAME,LATITUDE,LONGITUDE


Then let's loop through the data and fill the dataframe one row at a time.

In [5]:
for data in neighborhoods_data:
    area_id = data['properties']['AREA_ID']
    area_long_code = data['properties']['AREA_LONG_CODE']
    area_name = data['properties']['AREA_NAME']
    latitude = data['properties']['LATITUDE']
    longitude = data['properties']['LONGITUDE']
    neighborhoods = neighborhoods.append({'AREA_ID': area_id,
                            'AREA_LONG_CODE': area_long_code,
                            'AREA_NAME': area_name,
                            'LATITUDE': latitude,
                            'LONGITUDE': longitude}, ignore_index=True)

Quickly examine the resulting dataframe.

In [6]:
neighborhoods.head()

,AREA_ID,AREA_LONG_CODE,AREA_NAME,LATITUDE,LONGITUDE
0,25886861,94,Wychwood (94),43.676919,-79.425515
1,25886820,100,Yonge-Eglinton (100),43.704689,-79.403590
2,25886834,97,Yonge-St.Clair (97),43.687859,-79.397871
3,25886593,27,York University Heights (27),43.765736,-79.488883
4,25886688,31,Yorkdale-Glen Park (31),43.714672,-79.457108


In [7]:
neighborhoods = neighborhoods.drop_duplicates()
neighborhoods["AREA_NAME"]

0                                 Wychwood (94)
1                          Yonge-Eglinton (100)
2                           Yonge-St.Clair (97)
3                  York University Heights (27)
4                       Yorkdale-Glen Park (31)
5                      Lambton Baby Point (114)
6                         Lansing-Westgate (38)
7                     Lawrence Park North (105)
8                     Lawrence Park South (103)
9                       Leaside-Bennington (56)
10                         Little Portugal (84)
11                             Long Branch (19)
12                                Malvern (132)
13                              Maple Leaf (29)
14                           Markland Wood (12)
15                               Milliken (130)
16     Mimico (includes Humber Bay Shores) (17)
17                            Morningside (135)
18                               Moss Park (73)
19                           Mount Dennis (115)
20        Mount Olive-Silverstone-Jamest

#### Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>toronto_explorer</em>, as shown below.

In [8]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print ('The geographical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto City are 43.653963, -79.387207.


#### Create a map of New York with neighborhoods superimposed on top.

In [9]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lon, area_name in zip(neighborhoods['LATITUDE'], neighborhoods['LONGITUDE'], neighborhoods['AREA_NAME']):
        label = area_name
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_toronto)
map_toronto

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhood of Wychwood. So let's slice the original dataframe and create a new dataframe of the Wychwood data.

In [10]:
wychwood_data = neighborhoods[neighborhoods['AREA_ID'] == 25886861].reset_index(drop=True)
wychwood_data.head()

,AREA_ID,AREA_LONG_CODE,AREA_NAME,LATITUDE,LONGITUDE
0,25886861,94,Wychwood (94),43.676919,-79.425515


Let's get the geographical coordinates of Wychwood

In [11]:
address = 'Wychwood, Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Wychwood are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Wychwood are 43.6820944, -79.4238554.


As we did with all of New York City, let's visualizate Wychwood the neighborhoods in it.

In [12]:
map_wychwood = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lon, label in zip(wychwood_data['LATITUDE'], wychwood_data['LONGITUDE'], wychwood_data['AREA_NAME']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_wychwood)  
    
map_wychwood

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [13]:
# @hidden_cell
CLIENT_ID = 'JUZMOXW4YXHMROIURRKI5IPQVJ401DG34B2EIRIIQCSL4B2S' # 'your-client-ID' # your Foursquare ID
CLIENT_SECRET = 'HLEJVWI3UCNFGK1N2O5JSER4LVZISQQ0VORI04WUOUXJHI12' # 'your-client-secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)
print('CLIENT_ID: ' + 'client ID stored in variable CLIENT_ID')
print('CLIENT_SECRET:' + 'client SECRET stored in variable CLIENT_SECRET')

Your credentails:
CLIENT_ID: client ID stored in variable CLIENT_ID
CLIENT_SECRET:client SECRET stored in variable CLIENT_SECRET


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [14]:
wychwood_data.loc[0, 'AREA_NAME']

'Wychwood (94)'

Get the neighborhood's latitude and longitude values.

In [15]:
neighborhood_latitude = wychwood_data.loc[0, 'LATITUDE'] # neighborhood latitude value
neighborhood_longitude = wychwood_data.loc[0, 'LONGITUDE'] # neighborhood longitude value

neighborhood_name = wychwood_data.loc[0, 'AREA_NAME'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Wychwood (94) are 43.6769192679, -79.425514947.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [16]:
# @hidden_cell

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
# url # display URL this shows the client ID and SECRET
print ('Test URL properly set')



Test URL properly set


Double-click __here__ for the solution.
<!-- The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API
-->

<!--
radius = 500 # define radius
-->

<!--
\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
--> 

Send the GET request and examine the resutls

In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e1d280d9da7ee001b6ccb16'},
 'response': {'headerLocation': 'Bracondale Hill',
  'headerFullLocation': 'Bracondale Hill, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.6814192724, 'lng': -79.41930460333964},
   'sw': {'lat': 43.672419263399995, 'lng': -79.43172529066035}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b86e89df964a52051a531e3',
       'name': "Wychwood Barns Farmers' Market",
       'location': {'address': '601 Christie Street',
        'crossStreet': 'St Clair Avenue West',
        'lat': 43.68001040153905,
        'lng': -79.42384857341463,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.68001040153905,
          'lng': -79.4

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wychwood Barns Farmers' Market,Farmers Market,43.680010,-79.423849
1,Wychwood Barns,Event Space,43.680028,-79.423810
2,Hillcrest Park,Park,43.676012,-79.424787
3,Bob Coffee Bar,Coffee Shop,43.675376,-79.423268


And how many venues were returned by Foursquare?

In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto, for testing, first around Wychwood

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *wychwood_venues*.

In [22]:
wychwood_venues = getNearbyVenues(wychwood_data['AREA_NAME'],
                                   latitudes=wychwood_data['LATITUDE'],
                                   longitudes=wychwood_data['LONGITUDE']
                                  )
print(wychwood_venues)

Wychwood (94)
    Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Wychwood (94)              43.676919              -79.425515   
1  Wychwood (94)              43.676919              -79.425515   
2  Wychwood (94)              43.676919              -79.425515   
3  Wychwood (94)              43.676919              -79.425515   

                            Venue  Venue Latitude  Venue Longitude  \
0  Wychwood Barns Farmers' Market       43.680010       -79.423849   
1                  Wychwood Barns       43.680028       -79.423810   
2                  Hillcrest Park       43.676012       -79.424787   
3                  Bob Coffee Bar       43.675376       -79.423268   

   Venue Category  
0  Farmers Market  
1     Event Space  
2            Park  
3     Coffee Shop  


Double-click __here__ for the solution.
<!-- The correct answer is:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
--> 

#### Let's check the size of the resulting dataframe

In [23]:
print(wychwood_venues.shape)
wychwood_venues.head()

(4, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wychwood (94),43.676919,-79.425515,Wychwood Barns Farmers' Market,43.680010,-79.423849,Farmers Market
1,Wychwood (94),43.676919,-79.425515,Wychwood Barns,43.680028,-79.423810,Event Space
2,Wychwood (94),43.676919,-79.425515,Hillcrest Park,43.676012,-79.424787,Park
3,Wychwood (94),43.676919,-79.425515,Bob Coffee Bar,43.675376,-79.423268,Coffee Shop


Let's check how many venues were returned for each neighborhood

In [24]:
wychwood_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Wychwood (94),4,4,4,4,4,4


#### Let's find out how many unique categories can be curated from all the returned venues

In [25]:
print('There are {} uniques categories.'.format(len(wychwood_venues['Venue Category'].unique())))

There are 4 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [26]:
# one hot encoding
wychwood_onehot = pd.get_dummies(wychwood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
wychwood_onehot['Neighborhood'] = wychwood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [wychwood_onehot.columns[-1]] + list(wychwood_onehot.columns[:-1])
wychwood_onehot = wychwood_onehot[fixed_columns]

wychwood_onehot.head()

,Neighborhood,Coffee Shop,Event Space,Farmers Market,Park
0,Wychwood (94),0,0,1,0
1,Wychwood (94),0,1,0,0
2,Wychwood (94),0,0,0,1
3,Wychwood (94),1,0,0,0


And let's examine the new dataframe size.

In [27]:
wychwood_onehot.shape

(4, 5)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [28]:
wychwood_grouped = wychwood_onehot.groupby('Neighborhood').mean().reset_index()
wychwood_grouped

,Neighborhood,Coffee Shop,Event Space,Farmers Market,Park
0,Wychwood (94),0.25,0.25,0.25,0.25


#### Let's confirm the new size

In [29]:
wychwood_grouped.shape

(1, 5)

In [30]:
neighborhoods.shape

(140, 5)

In [31]:
# There are no child neighbourhoods for an area in this Toronto dataset
# All Toronto areas/neighborhoods are to be trated as equal brothers in category

import os.path

# if json is not locally saved, ask for it to the online service

if not os.path.isfile("toronto_venues.csv"):
    toronto_venues = getNearbyVenues(neighborhoods['AREA_NAME'],
                                   latitudes=neighborhoods['LATITUDE'],
                                   longitudes=neighborhoods['LONGITUDE']
                                  )

    toronto_venues.to_csv("toronto_venues.csv")
    print("saved local query back up in toronto_venues.csv file")
else:
    print("loaded local query back up from toronto_venues.csv file")
    toronto_venues = pd.read_csv("toronto_venues.csv") 
    
print(toronto_venues.shape)
toronto_venues.head()

loaded local query back up from toronto_venues.csv file
(2090, 8)


,Unnamed: 0,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,Wychwood (94),43.676919,-79.425515,Wychwood Barns Farmers' Market,43.680010,-79.423849,Farmers Market
1,1,Wychwood (94),43.676919,-79.425515,Wychwood Barns,43.680028,-79.423810,Event Space
2,2,Wychwood (94),43.676919,-79.425515,Hillcrest Park,43.676012,-79.424787,Park
3,3,Wychwood (94),43.676919,-79.425515,Bob Coffee Bar,43.675376,-79.423268,Coffee Shop
4,4,Yonge-Eglinton (100),43.704689,-79.403590,North Toronto Memorial Community Centre,43.706098,-79.404337,Gym


In [32]:
# Let's count how many venues are per neighborhood
toronto_venues.groupby('Neighborhood').count()

,Unnamed: 0,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
Agincourt North (129),19,19,19,19,19,19,19
Agincourt South-Malvern West (128),16,16,16,16,16,16,16
Alderwood (20),7,7,7,7,7,7,7
Annex (95),25,25,25,25,25,25,25
Banbury-Don Mills (42),22,22,22,22,22,22,22
Bathurst Manor (34),4,4,4,4,4,4,4
Bay Street Corridor (76),97,97,97,97,97,97,97
Bayview Village (52),4,4,4,4,4,4,4
Bayview Woods-Steeles (49),3,3,3,3,3,3,3


In [33]:
# Find how many unique categories are within the returned results
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 289 uniques categories.


In [34]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = list(toronto_onehot)
cols.insert(0, cols.pop(cols.index('Neighborhood')))
toronto_onehot = toronto_onehot.loc[:, cols]

toronto_onehot.head()

,Neighborhood,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Quad,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Swiss Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Transportation Service,Tree,Turkish Restaurant,Vegetari

In [35]:
toronto_onehot.shape

(2090, 289)

In [36]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Quad,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Swiss Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Transportation Service,Tree,Turkish Restaurant,Vegetari

In [37]:
toronto_grouped.shape

(137, 289)

#### Let's print each neighborhood along with the top 5 most common venues

In [38]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt North (129)----
                  venue  freq
0    Chinese Restaurant  0.11
1   Japanese Restaurant  0.05
2  Fast Food Restaurant  0.05
3          Liquor Store  0.05
4            Beer Store  0.05


----Agincourt South-Malvern West (128)----
                venue  freq
0  Chinese Restaurant  0.38
1           Pool Hall  0.06
2          Restaurant  0.06
3                Bank  0.06
4       Shopping Mall  0.06


----Alderwood (20)----
               venue  freq
0        Pizza Place  0.29
1  Convenience Store  0.14
2           Pharmacy  0.14
3        Coffee Shop  0.14
4             Market  0.14


----Annex (95)----
            venue  freq
0  Sandwich Place  0.12
1             Pub  0.08
2            Park  0.08
3            Café  0.08
4      Donut Shop  0.04


----Banbury-Don Mills (42)----
            venue  freq
0  Cosmetics Shop  0.09
1    Gourmet Shop  0.09
2     Coffee Shop  0.09
3      Shoe Store  0.09
4     Pizza Place  0.05


----Bathurst Manor (34)----
               ven

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt North (129),Chinese Restaurant,Pizza Place,Park,Sandwich Place,Japanese Restaurant,Bakery,Discount Store,Clothing Store,Fried Chicken Joint,Dim Sum Restaurant
1,Agincourt South-Malvern West (128),Chinese Restaurant,Pool Hall,Bank,Restaurant,Noodle House,Cantonese Restaurant,Seafood Restaurant,Shopping Mall,Filipino Restaurant,Malay Restaurant
2,Alderwood (20),Pizza Place,Convenience Store,Market,Pharmacy,Athletics & Sports,Coffee Shop,Farm,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
3,Annex (95),Sandwich Place,Café,Park,Pub,Cosmetics Shop,BBQ Joint,Social Club,Metro Station,Pharmacy,Middle Eastern Restaurant
4,Banbury-Don Mills (42),Shoe Store,Cosmetics Shop,Coffee Shop,Gourmet Shop,Bubble Tea Shop,Medical Center,Sporting Goods Shop,Movie Theater,Shopping Mall,Sandwich Place


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [41]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 1, 2, 2, 2, 2, 2, 3, 2], dtype=int32)

In [42]:
kmeans.labels_

array([2, 2, 1, 2, 2, 2, 2, 2, 3, 2, 1, 3, 2, 2, 2, 2, 2, 1, 3, 2, 1, 2,
       3, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 3, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2,
       3, 3, 2, 0, 2, 2, 2, 2, 3, 0, 2, 2, 2, 2, 1, 2, 1, 2, 2, 0, 3, 2,
       3, 3, 2, 2, 2, 2, 2, 1, 2, 1, 3, 2, 0, 2, 1, 0, 2, 2, 2, 2, 2, 3,
       2, 1, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 4, 1, 3, 2, 2, 2, 2, 2, 0, 2, 3, 2, 3, 2, 2, 2, 2, 3, 2, 2, 2,
       0, 2, 2, 2, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [43]:
# match column names
neighborhoods_venues_sorted.rename(columns={"Neighborhood":"AREA_NAME"}, inplace=True)
neighborhoods_venues_sorted

,AREA_NAME,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt North (129),Chinese Restaurant,Pizza Place,Park,Sandwich Place,Japanese Restaurant,Bakery,Discount Store,Clothing Store,Fried Chicken Joint,Dim Sum Restaurant
1,Agincourt South-Malvern West (128),Chinese Restaurant,Pool Hall,Bank,Restaurant,Noodle House,Cantonese Restaurant,Seafood Restaurant,Shopping Mall,Filipino Restaurant,Malay Restaurant
2,Alderwood (20),Pizza Place,Convenience Store,Market,Pharmacy,Athletics & Sports,Coffee Shop,Farm,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
3,Annex (95),Sandwich Place,Café,Park,Pub,Cosmetics Shop,BBQ Joint,Social Club,Metro Station,Pharmacy,Middle Eastern Restaurant
4,Banbury-Don Mills (42),Shoe Store,Cosmetics Shop,Coffee Shop,Gourmet Shop,Bubble Tea Shop,Medical Center,Sporting Goods Shop,Movie Theater,Shopping Mall,Sandwich Place
5,Bathurst Manor (34),Convenience Store,Playground,Park,Baseball Field,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Egyptian Restaurant
6,Bay Street Corridor (76),Coffee Shop,Burger Joint,Clothing Store,Café,Sandwich Place,Italian Restaurant,Middle Eastern Restaurant,Restaurant,Salad Place,Juice Bar
7,Bayview Village (52),Trail,Dog Run,Golf Driving Range,Construction & Landscaping,Cuban Restaurant,Egyptian Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant
8,Bayview Woods-Steeles (49),Basketball Court,Dog Run,Park,Zoo Exhibit,Falafel Restaurant,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Farm
9,Bedford Park-Nortown (39),Italian Restaurant,Coffee Shop,Fast Food Restaurant,Sandwich Place,Pizza Place,Café,Liquor Store,Sushi Restaurant,Juice Bar,Thai Restaurant


In [44]:
neighborhoods_venues_sorted.shape

(137, 11)

In [45]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [46]:
neighborhoods_venues_sorted.shape

(137, 12)

In [47]:
neighborhoods_venues_sorted

,Cluster Labels,AREA_NAME,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,Agincourt North (129),Chinese Restaurant,Pizza Place,Park,Sandwich Place,Japanese Restaurant,Bakery,Discount Store,Clothing Store,Fried Chicken Joint,Dim Sum Restaurant
1,2,Agincourt South-Malvern West (128),Chinese Restaurant,Pool Hall,Bank,Restaurant,Noodle House,Cantonese Restaurant,Seafood Restaurant,Shopping Mall,Filipino Restaurant,Malay Restaurant
2,1,Alderwood (20),Pizza Place,Convenience Store,Market,Pharmacy,Athletics & Sports,Coffee Shop,Farm,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
3,2,Annex (95),Sandwich Place,Café,Park,Pub,Cosmetics Shop,BBQ Joint,Social Club,Metro Station,Pharmacy,Middle Eastern Restaurant
4,2,Banbury-Don Mills (42),Shoe Store,Cosmetics Shop,Coffee Shop,Gourmet Shop,Bubble Tea Shop,Medical Center,Sporting Goods Shop,Movie Theater,Shopping Mall,Sandwich Place
5,2,Bathurst Manor (34),Convenience Store,Playground,Park,Baseball Field,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Egyptian Restaurant
6,2,Bay Street Corridor (76),Coffee Shop,Burger Joint,Clothing Store,Café,Sandwich Place,Italian Restaurant,Middle Eastern Restaurant,Restaurant,Salad Place,Juice Bar
7,2,Bayview Village (52),Trail,Dog Run,Golf Driving Range,Construction & Landscaping,Cuban Restaurant,Egyptian Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant
8,3,Bayview Woods-Steeles (49),Basketball Court,Dog Run,Park,Zoo Exhibit,Falafel Restaurant,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Farm
9,2,Bedford Park-Nortown (39),Italian Restaurant,Coffee Shop,Fast Food Restaurant,Sandwich Place,Pizza Place,Café,Liquor Store,Sushi Restaurant,Juice Bar,Thai Restaurant


In [48]:
toronto_merged = neighborhoods # toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('AREA_NAME'), on='AREA_NAME')

In [49]:
toronto_merged.shape

(140, 16)

In [50]:
toronto_merged

# We can see in the full table that some areas/neighborhoods do not have venues and then can't be assigned to a cluster with the same criteria than others

,AREA_ID,AREA_LONG_CODE,AREA_NAME,LATITUDE,LONGITUDE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,25886861,94,Wychwood (94),43.676919,-79.425515,0.0,Coffee Shop,Park,Event Space,Farmers Market,Farm,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Exhibit,Falafel Restaurant
1,25886820,100,Yonge-Eglinton (100),43.704689,-79.403590,2.0,Coffee Shop,Fast Food Restaurant,Gym,Buffet,Sandwich Place,Supermarket,Steakhouse,Breakfast Spot,Burger Joint,Smoothie Shop
2,25886834,97,Yonge-St.Clair (97),43.687859,-79.397871,2.0,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pharmacy,Pizza Place,Sandwich Place,Café,Pub,Gym,Grocery Store
3,25886593,27,York University Heights (27),43.765736,-79.488883,2.0,Bank,Falafel Restaurant,Bar,Massage Studio,Pizza Place,Metro Station,Japanese Restaurant,Caribbean Restaurant,Fast Food Restaurant,Coffee Shop
4,25886688,31,Yorkdale-Glen Park (31),43.714672,-79.457108,2.0,Fast Food Restaurant,Restaurant,Athletics & Sports,Furniture / Home Store,Italian Restaurant,Bank,Bookstore,Seafood Restaurant,Coffee Shop,Greek Restaurant
5,25886404,114,Lambton Baby Point (114),43.657420,-79.496045,3.0,Park,River,Zoo Exhibit,Exhibit,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant
6,25886627,38,Lansing-Westgate (38),43.754271,-79.424748,3.0,Park,Soccer Field,Zoo Exhibit,Falafel Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Farm
7,25886808,105,Lawrence Park North (105),43.730060,-79.403978,2.0,Sushi Restaurant,Bakery,Italian Restaurant,Pub,Burger Joint,Sandwich Place,Asian Restaurant,Cosmetics Shop,Coffee Shop,Restaurant
8,25886812,103,Lawrence Park South (103),43.717212,-79.406039,2.0,Sporting Goods Shop,Gym / Fitness Center,Flower Shop,Spa,Chinese Restaurant,Coffee Shop,Rental Car Location,Dessert Shop,Clothing Store,Falafel Restaurant
9,25886328,56,Leaside-Bennington (56),43.703797,-79.366072,2.0,Coffee Shop,Pool,Italian Restaurant,Performing Arts Venue,Park,Dessert Shop,Sandwich Place,Shopping Mall,Skating Rink,Mexican Restaurant


In [51]:
# The neighbourhoods (areas) that did not report any venue don't belong to any category
# Areas without venues will not be considered for this study
toronto_merged.dropna(inplace= True)

toronto_merged

,AREA_ID,AREA_LONG_CODE,AREA_NAME,LATITUDE,LONGITUDE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,25886861,94,Wychwood (94),43.676919,-79.425515,0.0,Coffee Shop,Park,Event Space,Farmers Market,Farm,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Exhibit,Falafel Restaurant
1,25886820,100,Yonge-Eglinton (100),43.704689,-79.403590,2.0,Coffee Shop,Fast Food Restaurant,Gym,Buffet,Sandwich Place,Supermarket,Steakhouse,Breakfast Spot,Burger Joint,Smoothie Shop
2,25886834,97,Yonge-St.Clair (97),43.687859,-79.397871,2.0,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pharmacy,Pizza Place,Sandwich Place,Café,Pub,Gym,Grocery Store
3,25886593,27,York University Heights (27),43.765736,-79.488883,2.0,Bank,Falafel Restaurant,Bar,Massage Studio,Pizza Place,Metro Station,Japanese Restaurant,Caribbean Restaurant,Fast Food Restaurant,Coffee Shop
4,25886688,31,Yorkdale-Glen Park (31),43.714672,-79.457108,2.0,Fast Food Restaurant,Restaurant,Athletics & Sports,Furniture / Home Store,Italian Restaurant,Bank,Bookstore,Seafood Restaurant,Coffee Shop,Greek Restaurant
5,25886404,114,Lambton Baby Point (114),43.657420,-79.496045,3.0,Park,River,Zoo Exhibit,Exhibit,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant
6,25886627,38,Lansing-Westgate (38),43.754271,-79.424748,3.0,Park,Soccer Field,Zoo Exhibit,Falafel Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Farm
7,25886808,105,Lawrence Park North (105),43.730060,-79.403978,2.0,Sushi Restaurant,Bakery,Italian Restaurant,Pub,Burger Joint,Sandwich Place,Asian Restaurant,Cosmetics Shop,Coffee Shop,Restaurant
8,25886812,103,Lawrence Park South (103),43.717212,-79.406039,2.0,Sporting Goods Shop,Gym / Fitness Center,Flower Shop,Spa,Chinese Restaurant,Coffee Shop,Rental Car Location,Dessert Shop,Clothing Store,Falafel Restaurant
9,25886328,56,Leaside-Bennington (56),43.703797,-79.366072,2.0,Coffee Shop,Pool,Italian Restaurant,Performing Arts Venue,Park,Dessert Shop,Sandwich Place,Shopping Mall,Skating Rink,Mexican Restaurant


In [52]:
# recast to int32 the label column after join operation
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

toronto_merged

,AREA_ID,AREA_LONG_CODE,AREA_NAME,LATITUDE,LONGITUDE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,25886861,94,Wychwood (94),43.676919,-79.425515,0,Coffee Shop,Park,Event Space,Farmers Market,Farm,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Exhibit,Falafel Restaurant
1,25886820,100,Yonge-Eglinton (100),43.704689,-79.403590,2,Coffee Shop,Fast Food Restaurant,Gym,Buffet,Sandwich Place,Supermarket,Steakhouse,Breakfast Spot,Burger Joint,Smoothie Shop
2,25886834,97,Yonge-St.Clair (97),43.687859,-79.397871,2,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pharmacy,Pizza Place,Sandwich Place,Café,Pub,Gym,Grocery Store
3,25886593,27,York University Heights (27),43.765736,-79.488883,2,Bank,Falafel Restaurant,Bar,Massage Studio,Pizza Place,Metro Station,Japanese Restaurant,Caribbean Restaurant,Fast Food Restaurant,Coffee Shop
4,25886688,31,Yorkdale-Glen Park (31),43.714672,-79.457108,2,Fast Food Restaurant,Restaurant,Athletics & Sports,Furniture / Home Store,Italian Restaurant,Bank,Bookstore,Seafood Restaurant,Coffee Shop,Greek Restaurant
5,25886404,114,Lambton Baby Point (114),43.657420,-79.496045,3,Park,River,Zoo Exhibit,Exhibit,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant
6,25886627,38,Lansing-Westgate (38),43.754271,-79.424748,3,Park,Soccer Field,Zoo Exhibit,Falafel Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Farm
7,25886808,105,Lawrence Park North (105),43.730060,-79.403978,2,Sushi Restaurant,Bakery,Italian Restaurant,Pub,Burger Joint,Sandwich Place,Asian Restaurant,Cosmetics Shop,Coffee Shop,Restaurant
8,25886812,103,Lawrence Park South (103),43.717212,-79.406039,2,Sporting Goods Shop,Gym / Fitness Center,Flower Shop,Spa,Chinese Restaurant,Coffee Shop,Rental Car Location,Dessert Shop,Clothing Store,Falafel Restaurant
9,25886328,56,Leaside-Bennington (56),43.703797,-79.366072,2,Coffee Shop,Pool,Italian Restaurant,Performing Arts Venue,Park,Dessert Shop,Sandwich Place,Shopping Mall,Skating Rink,Mexican Restaurant


Finally, let's visualize the resulting clusters

In [53]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['LATITUDE'], toronto_merged['LONGITUDE'], toronto_merged['AREA_NAME'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,AREA_LONG_CODE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,94,0,Coffee Shop,Park,Event Space,Farmers Market,Farm,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Exhibit,Falafel Restaurant
17,135,0,Sandwich Place,Coffee Shop,Park,General Travel,Beer Store,Supermarket,Pharmacy,Fast Food Restaurant,Discount Store,Zoo Exhibit
20,2,0,Coffee Shop,Japanese Restaurant,Park,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant
41,111,0,Park,Pizza Place,Convenience Store,Bus Line,Beer Store,Coffee Shop,Asian Restaurant,Supermarket,Field,Filipino Restaurant
60,43,0,Portuguese Restaurant,Pizza Place,French Restaurant,Intersection,Park,Coffee Shop,Zoo Exhibit,Exhibit,Electronics Store,Empanada Restaurant
103,59,0,Convenience Store,Athletics & Sports,Italian Restaurant,Park,Farmers Market,Coffee Shop,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
114,11,0,Convenience Store,Pizza Place,Park,Coffee Shop,Chinese Restaurant,Hockey Arena,Cupcake Shop,Eastern European Restaurant,Fish & Chips Shop,Filipino Restaurant
121,140,0,Sandwich Place,Pizza Place,Sports Bar,Park,Coffee Shop,Hotel,Field,Fast Food Restaurant,Farmers Market,Farm
127,8,0,Pizza Place,Park,Zoo Exhibit,Farm,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant
137,6,0,Coffee Shop,Grocery Store,Pizza Place,Farm,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant


#### Cluster 2

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,AREA_LONG_CODE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,132,1,Bubble Tea Shop,Pizza Place,Sandwich Place,Pharmacy,Fried Chicken Joint,Fast Food Restaurant,Zoo Exhibit,Exhibit,Electronics Store,Empanada Restaurant
14,12,1,Pizza Place,Discount Store,Fast Food Restaurant,Bank,Zoo Exhibit,Falafel Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
19,115,1,Asian Restaurant,Fast Food Restaurant,American Restaurant,Convenience Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm
29,54,1,Pizza Place,Gastropub,Intersection,Pharmacy,Pet Store,Breakfast Spot,Fast Food Restaurant,Gym / Fitness Center,Athletics & Sports,Bank
53,118,1,Pizza Place,Pharmacy,Convenience Store,Bus Stop,Thai Restaurant,Bank,Italian Restaurant,Chinese Restaurant,Shopping Mall,Noodle House
76,20,1,Pizza Place,Convenience Store,Market,Pharmacy,Athletics & Sports,Coffee Shop,Farm,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
84,112,1,Museum,Fast Food Restaurant,Zoo Exhibit,Farm,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant
91,57,1,Clothing Store,Theater,Pizza Place,Discount Store,Zoo Exhibit,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant
94,109,1,Women's Store,Market,Park,Fast Food Restaurant,Zoo Exhibit,Falafel Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
100,123,1,Fast Food Restaurant,Burger Joint,Pizza Place,Hardware Store,Park,Zoo Exhibit,Falafel Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant


#### Cluster 3

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,AREA_LONG_CODE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,100,2,Coffee Shop,Fast Food Restaurant,Gym,Buffet,Sandwich Place,Supermarket,Steakhouse,Breakfast Spot,Burger Joint,Smoothie Shop
2,97,2,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pharmacy,Pizza Place,Sandwich Place,Café,Pub,Gym,Grocery Store
3,27,2,Bank,Falafel Restaurant,Bar,Massage Studio,Pizza Place,Metro Station,Japanese Restaurant,Caribbean Restaurant,Fast Food Restaurant,Coffee Shop
4,31,2,Fast Food Restaurant,Restaurant,Athletics & Sports,Furniture / Home Store,Italian Restaurant,Bank,Bookstore,Seafood Restaurant,Coffee Shop,Greek Restaurant
7,105,2,Sushi Restaurant,Bakery,Italian Restaurant,Pub,Burger Joint,Sandwich Place,Asian Restaurant,Cosmetics Shop,Coffee Shop,Restaurant
8,103,2,Sporting Goods Shop,Gym / Fitness Center,Flower Shop,Spa,Chinese Restaurant,Coffee Shop,Rental Car Location,Dessert Shop,Clothing Store,Falafel Restaurant
9,56,2,Coffee Shop,Pool,Italian Restaurant,Performing Arts Venue,Park,Dessert Shop,Sandwich Place,Shopping Mall,Skating Rink,Mexican Restaurant
10,84,2,Café,Bar,Coffee Shop,Restaurant,Korean Restaurant,Cocktail Bar,Bakery,Park,Thai Restaurant,Italian Restaurant
11,19,2,Coffee Shop,Grocery Store,Bar,Italian Restaurant,Restaurant,Café,Greek Restaurant,Beer Store,Wings Joint,Pharmacy
13,29,2,Gym / Fitness Center,Convenience Store,Trail,Chinese Restaurant,Coffee Shop,Zoo Exhibit,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space


#### Cluster 4

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,AREA_LONG_CODE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,114,3,Park,River,Zoo Exhibit,Exhibit,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant
6,38,3,Park,Soccer Field,Zoo Exhibit,Falafel Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Farm
15,130,3,Park,Sandwich Place,Zoo Exhibit,Exhibit,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant
27,68,3,Park,Café,Pool,Dog Run,Exhibit,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant
34,45,3,Food & Drink Shop,Park,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm
54,61,3,Theater,Park,Zoo Exhibit,Falafel Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Farm
62,136,3,Park,Construction & Landscaping,Gym / Fitness Center,Exhibit,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant
64,35,3,Construction & Landscaping,Baby Store,Park,Falafel Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Farm
69,37,3,Mobile Phone Shop,Park,Zoo Exhibit,Farm,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market
82,49,3,Basketball Court,Dog Run,Park,Zoo Exhibit,Falafel Restaurant,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Farm


#### Cluster 5

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,AREA_LONG_CODE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,16,4,Baseball Field,Zoo Exhibit,Food Court,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market


### Thank you for reviewing this lab!

This lab is a task for the Coursera specialization "Data Science" organized by IBM.

Author: [Santiago Maraggi](http://www.linkedin.com/in/santiagomaraggi/)

This notebook is inspired in the one of reference provided by [Alex Aklson](https://www.linkedin.com/in/aklson/) and [Polong Lin](https://www.linkedin.com/in/polonglin/). I hope you found this lab interesting and educational. Feel free to contact us if you have any questions!

This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).

<hr>

Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).